<a href="https://colab.research.google.com/github/felixyustian/chatterbot-corpus/blob/master/Langchain_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Materi ini di buat oleh : [Sardi Irfansyah](https://www.linkedin.com/in/sirfansyah/)**

# Instalasi Library

In [ ]:
!pip install -U -q google-generativeai langchain langchain-google-genai langchain_community pypdf chromadb google-search-results

In [ ]:
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from IPython.display import Markdown as md

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata

In [ ]:
# masukan API key kalian
api_key= userdata.get('GEMINI_API_KEY')

## Basic Langchain (Chain)
- Building block yang paling simple
- LLMs, Prompt, Output Parser
- tidka bisa menggunakan external tools (web search, search database)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Komponen yang pertama yaitu LLMs
# Cara pertama
chat_model = ChatGoogleGenerativeAI(google_api_key=api_key,
                                   model="gemini-2.0-flash")

In [ ]:
# chat_model = ChatGoogleGenerativeAI(google_api_key="abcdefalasjdljdaljdlasjdds",
#                                    model="gemini-2.0-flash")

In [ ]:
#Komponen yang kedua
system_instruction = """
Kamu adalah guru matematika, tugasmu membantu siswa untuk menjawab pertanyaan soal matematika.
jawab secara detail, step by step dan jelaskan setiap langkah yang kamu lakukan.
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_instruction),
    ("user", "{input}")
])

In [ ]:
#Komponen yang ketiga yaitu output parser
output_parser = StrOutputParser()

In [ ]:
#Kita gabungkan semuanya
chain = prompt | chat_model | output_parser

In [ ]:
response = chain.invoke({"input": "berapa jarak yang ditempuh dipta ketika melaju dengan kecepatan 100km/30 menit selama 2 jam ?"})

In [ ]:
md(response)

Baik, mari kita pecahkan soal ini langkah demi langkah:

**1. Memahami Kecepatan**

*   Kecepatan Dipta adalah 100 km/30 menit. Ini berarti dalam 30 menit, dia menempuh jarak 100 km.

**2. Menghitung Jarak per Jam**

*   Karena 1 jam ada 60 menit, maka 1 jam sama dengan 2 kali 30 menit.
*   Jadi, dalam 1 jam, Dipta menempuh jarak: 100 km/30 menit * 2 = 200 km/jam

**3. Menghitung Jarak Total**

*   Dipta berkendara selama 2 jam dengan kecepatan 200 km/jam.
*   Maka, jarak total yang ditempuh adalah: 200 km/jam * 2 jam = 400 km

**Jawaban:**

Jadi, jarak yang ditempuh Dipta selama 2 jam adalah 400 km.

## Structured Output

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
"""
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    language: str = Field(description="The language the text is written in")

In [ ]:
llm = ChatGoogleGenerativeAI(google_api_key=api_key,
                                   model="gemini-2.0-flash").with_structured_output(
    Classification
)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)
response

Classification(sentiment='Positive', language='Spanish')

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': 'negative', 'language': 'Spanish'}

In [ ]:
inp = "wahh hari ini cuaca sangat mendukung utnuk bermain bola"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': 'positive', 'language': 'indonesian'}

## Mid Langchain (Agent)
- lebih punya banyak capabilities dari `Chain`
- Strukturnya hampir sama dengan `Chain`
- Yang membedakan ada 2
  - Agent menggunakan framework ReAct secara default
  - Agent bisa menggunakan external tools (Calculator, Search Internet)

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chains import LLMMathChain #Untuk kalkulator
from langchain.agents import Tool

os.environ["SERPAPI_API_KEY"] = userdata.get('SERP_API_KEY')

In [ ]:
# Membuat tools yang pertama yaitu kalkulator
chat_model = ChatGoogleGenerativeAI(google_api_key=api_key,
                                   model="gemini-2.0-flash")

problem_chain = LLMMathChain.from_llm(llm=chat_model)

math_tool = Tool.from_function(name="Calculator",
                               func=problem_chain.run,
                               description="""Useful for when you need to answer questions
                               about math. This tool is only for math questions and nothing else. Only input
                               math expressions.""")

In [ ]:
tools = load_tools(["serpapi"])

In [ ]:
tools.append(math_tool)

In [ ]:
# initialize agent
agent = initialize_agent(
    tools,
    chat_model,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# invoke agent
agent( """I have 3 apples and 4 oranges. I give half of my oranges
               away and buy two dozen new ones, alongwith three packs of
               strawberries. Each pack of strawberry has 30 strawberries.
               How  many total pieces of fruit do I have at the end?""")



> Entering new AgentExecutor chain...
Thought: First, I need to calculate how many oranges I have after giving half away and buying two dozen. Then, I need to calculate how many strawberries I have after buying three packs. Finally, I need to add the number of apples, oranges, and strawberries to find the total number of pieces of fruit.

Action:
```json
{
  "action": "Calculator",
  "action_input": "4/2"
}
```
Observation: Answer: 2.0
Thought:I need to calculate how many oranges I have after giving half away and buying two dozen. Then, I need to calculate how many strawberries I have after buying three packs. Finally, I need to add the number of apples, oranges, and strawberries to find the total number of pieces of fruit.

Action:
```json
{
  "action": "Calculator",
  "action_input": "4/2"
}
```
Observation: Answer: 2.0
Thought:

Thought: First, I need to calculate how many oranges I have after giving half away and buying two dozen. Then, I need to calculate how many strawberries I have after buying three packs. Finally, I need to add the number of apples, oranges, and strawberries to find the total number of pieces of fruit.

Action:
```json
{
  "action": "Calculator",
  "action_input": "4/2"
}
```

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
# invoke agent
agent("the difference between a stock and a bond")



> Entering new AgentExecutor chain...
Thought: I need to find the definitions of stocks and bonds and highlight their differences.
Action:
```json
{
  "action": "Search",
  "action_input": "difference between stock and bond"
}
```
Observation: ['Stocks represent ownership in a company, while bonds are essentially loans to a company or government, offering fixed interest payments and principal repayment at maturity.', ' entity_type: related_questions.', 'Stocks are ownership shares in a company, while bonds are a kind of loan from investors to a company or government. To make a profit from stocks, ...', 'Feb 20, 2025 —', 'Oct 30, 2024 —', 'The bond market is where investors go to trade debt securities, while the stock market is where investors trade equity securities through stock exchanges.', "What's the difference between the two? · A stock amounts to a piece of ownership in a company. · A bond, on the other hand, is like a loan.", 'Of course, the two asset classes provide different

{'input': 'the difference between a stock and a bond',
 'output': 'Stocks represent ownership in a company, whereas bonds are loans to a company or government. Stocks offer the potential for capital growth but are higher risk, while bonds provide a regular income stream with lower risk. Bonds generally provide more predictable returns than stocks.'}

In [ ]:
# invoke agent
agent("why nvidia stocks are up?")



> Entering new AgentExecutor chain...
Thought: I need to find out the reasons behind the increase in Nvidia stock prices. I will use a search engine to find relevant news articles and financial analysis reports.
Action:
```json
{
  "action": "Search",
  "action_input": "why is nvidia stock up"
}
```
Observation: New AI Chips Impress Wall Street, but Some Doubts Remain. stock was gaining Thursday as investors evaluated news from its GTC conference. Confidence in the superiority of its artificial-intelligence chips is being weighed against doubts over highly valued tech stocks.
Thought:I need to find more specific reasons. The previous search result indicates AI chips and the GTC conference as factors, but I need more details.
Action:
```json
{
  "action": "Search",
  "action_input": "nvidia stock up GTC conference"
}
```
Observation: [{'title': "As Nvidia's GTC wraps up, what's next for AI, quantum computing?", 'link': 'https://finance.yahoo.com/video/nvidias-gtc-wraps-whats-next-1415

{'input': 'why nvidia stocks are up?',
 'output': "Nvidia's stock is up due to impressive new AI chips revealed at the GTC conference and overall confidence in Nvidia's leadership in the AI market."}

In [ ]:
# invoke agent
agent("capital of france")



> Entering new AgentExecutor chain...
Thought: I need to find the capital of France. This is a well-known fact, so I can directly answer it.
Final Answer: Paris

> Finished chain.


{'input': 'capital of france', 'output': 'Paris'}

# Muali Sesi 2

## Advanced Langchain with RAG

## Langkah 1 - Siapkan Data

In [ ]:
# download pdf dengan curl
!curl -o  ai_pv.pdf https://www.pearsonvue.com/content/dam/VUE/vue/en/documents/clients/it-specialist/its-od-307-artificial-intel-pearson.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  259k  100  259k    0     0   749k      0 --:--:-- --:--:-- --:--:--  750k


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader

loader = PyPDFLoader("ai_pv.pdf")
pages = loader.load_and_split()

In [ ]:
web_loader = WebBaseLoader("https://instiki.ac.id/2023/08/14/data-scientist-jadi-incaran-instansi-pemerintahan-inilah-peluang-karier-dalam-cpns-2023/")
web_pages = web_loader.load_and_split()

In [ ]:
pages[0].page_content

'Copyright © 2025 Pearson Education, Inc. or its affiliates(s). All rights reserved.\n1. AI Problem Definition\n1.1 Identify the problem you are trying to solve using AI (e.g., user \nsegmentation, improving customer service)\n• Identify the need that will be addressed\n• Find out what information comes in and what output is expected\n• Determine whether AI is called for\n• Consider upsides and downsides of AI in the situation\n• Define measurable success\n• Benchmark against domain or organization-specific risks to which the \nproject may be susceptible\n1.2 Classify the problem (e.g., regression, unsupervised learning) \n• Examine available data (labeled or unlabeled?) and the problem\n• Determine problem type (e.g., classification, regression, unsupervised, \nreinforcement)\n1.3 Identify the areas of expertise needed to solve the problem\n• Identify business expertise required\n• Identify the need for domain (subject-matter) expertise on the problem\n• Identify AI expertise needed\n

In [ ]:
len(pages)

5

In [ ]:
md(web_pages[1].page_content)

Search


 Search












  





















Data Scientist Jadi Incaran Instansi Pemerintahan? Inilah Peluang Karier dalam CPNS 2023! 























Data Scientist Jadi Profesi Incaran Instansi Pemerintahan? Inilah Peluang Karier dalam CPNS 2023!
Civitas akademika, yuk bedah peluang profesi data science selain di startup dan IT company, yakni di instansi pemerintahan. Memangnya semenarik itu?
Bagi kamu yang tertarik berkarier jadi data science, pastinya pernah berpikir bahwa menjadi data scientist hanya terbatas pada startup atau perusahaan IT. Namun, kenyataannya saat ini banyak instansi pemerintahan yang mencari talenta digital dengan berbagai keahlian, termasuk data scientist. Tidak percaya? Ayo kita lihat lebih detail.
Menurut menpan.go.id, salah satu prioritas dalam penerimaan CPNS 2023 adalah data scientist. Hal ini disebabkan oleh perubahan cepat dalam dunia digital, yang memaksa pemerintah untuk beradaptasi dengan cepat agar tidak tertinggal. Perekrutan data scientist juga membantu mengurangi dampak transformasi digital pada rekrutmen jabatan lainnya.
Kementerian Keuangan Republik Indonesia juga menyatakan kebutuhan mereka akan sumber daya manusia yang ahli di bidang data science, khususnya di Direktorat Penilaian. Menurut KEMENKEU, data scientist di Direktorat Penilaian akan berperan dalam mengolah big data menggunakan berbagai alat untuk mengumpulkan, menginterpretasi, dan menganalisis data, sehingga menghasilkan wawasan yang berharga.
Bukan hanya KEMENKEU, banyak instansi pemerintah lain yang sedang mencari data scientist. Sebagai contoh, Direktorat Pengembangan UMKM dan Koperasi dari Kementerian PPN/Bappenas membuka lowongan untuk data scientist.
Mengapa Data Scientist Kini Jadi Target Utama Instansi Pemerintahan?
Jika sebelumnya pemerintah bekerja dengan data secara manual, kehadiran data scientist memungkinkan pemerintah untuk mengolah data secara efektif, meningkatkan kinerja, melindungi dari serangan siber, dan bahkan meningkatkan pendapatan negara. Dampak positif ini pada akhirnya juga dirasakan oleh masyarakat.
Penerapan data science digunakan di berbagai sektor, beberapa contohnya sebagai berikut:
* Infrastruktur: Menganalisis dan mengatasi risiko dalam pembangunan jalan tol.
* Pariwisata: Pemetaan lokasi yang tepat untuk wisatawan dan strategi.
* pemasaran berdasarkan segmentasi kelompok.
* Pertanian: Pengelolaan hama, penanganan perubahan cuaca, prediksi hasil produksi.
* Perdagangan: Peramalan bantuan sosial atau kebutuhan sembako.
* Kesehatan: Peramalan penyebaran penyakit.
Jadi, peluang menjadi data scientist tidak hanya terbatas pada startup atau perusahaan IT, tetapi juga sangat terbuka di instansi pemerintahan. Kamu bisa memilih sektor pemerintahan yang paling menarik bagi kamu dan menjadi talenta dihital yang mendukung transformasi digital pemerintah. Kamu dapat memulainya dengan belajar skill data scientist di Institut Bisnis dan Teknologi Indonesia (INSTIKI), sebuah kampus IT, bisnis, dan desain terkemuka di Bali dan Nusa Tenggara yang telah terakreditasi secara nasional dan tersertifikasi internasional. Di INSTIKI, kamu akan dibimbing oleh dosen-dosen berpengalaman dengan kurikulum yang sesuai dengan kebutuhan industri! Untuk informasi lebih lengkap dapat mengakses website INSTIKI.












PENGUMUMAN LAINNYA 















Telusuri Artikel Lainnya 



 







				INSTIKI Gelar Wisuda XXV, Lahirkan Sarjana Unggul & Berdaya Saing			



			22 March 2025		










				Kegiatan Sederhana Ini Bikin Portofoliomu Jadi Lebih Menarik			



			20 March 2025		










				Magical Farewell Party: INSTIKI Lepas Calon Wisudawan dengan Konsep Disney			



			20 March 2025		




















 







 















 






 


Instagram
 



Facebook-square
 



Twitter
 



Youtube
 










							INSTITUT BISNIS DAN TEKNOLOGI INDONESIA						



							Senin – Sabtu | 08.00 – 22.00 WITA

## Langkah ke 2 - Chunking

In [ ]:
!pip install -U -q nltk

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Agar lebih terbanyang tentang penerapan chunk, kita akan uji coba terlebih dahulu dengan text sederhana.

In [ ]:
from langchain_text_splitters import NLTKTextSplitter, RecursiveCharacterTextSplitter

# Misalkan kamu memiliki dokumen seperti berikut
simple_doc = """
halo nama saya sardi irfansyah, saya lahir di jakarta.
Saya irfan. tinggal di Jakarta. saya sangat suka belajar AI terutama di bidang NLP dan CV
"""

print('panjang total karakter:',len(simple_doc),'\n')

panjang total karakter: 146 



In [ ]:
# Membuat objek NLTKTextSplitter dengan ukuran chunk dan overlap
text_splitter = NLTKTextSplitter(separator='\n\n',
                                 chunk_size=50,
                                 chunk_overlap=20) #default separator='\n\n'
#chunk 1 jumlahnya 50 char
#chunk 2 jumlahnya 37 char

In [ ]:
# Memecah dokumen menjadi beberapa chunk
chunks = text_splitter.split_text(simple_doc)
print(chunks,'\n')

# Menampilkan hasil chunk
for i, chunk in enumerate(chunks):
    #panjang karakter
    print(f"Panjang chunk {i+1}: {len(chunk)} karakter")
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 50)

['halo nama saya sardi irfansyah, saya lahir di jakarta.', 'Saya irfan.\n\ntinggal di Jakarta.', 'saya sangat suka belajar AI terutama di bidang NLP dan CV'] 

Panjang chunk 1: 54 karakter
Chunk 1:
halo nama saya sardi irfansyah, saya lahir di jakarta.
--------------------------------------------------
Panjang chunk 2: 32 karakter
Chunk 2:
Saya irfan.

tinggal di Jakarta.
--------------------------------------------------
Panjang chunk 3: 57 karakter
Chunk 3:
saya sangat suka belajar AI terutama di bidang NLP dan CV
--------------------------------------------------


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=20,
    is_separator_regex=False,
    keep_separator=False
)

In [ ]:
# Memecah dokumen menjadi beberapa chunk
chunks = text_splitter.split_text(simple_doc)
print(chunks,'\n')

# Menampilkan hasil chunk
for i, chunk in enumerate(chunks):
    #panjang karakter
    print(f"Panjang chunk {i+1}: {len(chunk)} karakter")
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 50)

['halo nama saya sardi irfansyah, saya lahir di', 'saya lahir di jakarta.', 'Saya irfan. tinggal di Jakarta. saya sangat suka', 'saya sangat suka belajar AI terutama di bidang NLP', 'di bidang NLP dan CV'] 

Panjang chunk 1: 45 karakter
Chunk 1:
halo nama saya sardi irfansyah, saya lahir di
--------------------------------------------------
Panjang chunk 2: 22 karakter
Chunk 2:
saya lahir di jakarta.
--------------------------------------------------
Panjang chunk 3: 48 karakter
Chunk 3:
Saya irfan. tinggal di Jakarta. saya sangat suka
--------------------------------------------------
Panjang chunk 4: 50 karakter
Chunk 4:
saya sangat suka belajar AI terutama di bidang NLP
--------------------------------------------------
Panjang chunk 5: 20 karakter
Chunk 5:
di bidang NLP dan CV
--------------------------------------------------


Penjelasan:
- Dapat kita lihat bahwa `NLTKTextSplitter` akan mencoba membuat text tersebut dipisahkan berdasarkan kalimat atau tanda `titik`. Jadi setiap ada titik maka akan dibuat chunk.
- Ketika panjang karakter lebih dari `chunk_size`, ini akan mengakibatkan peringatan warning.
- `chunk_overlap` digunakan untuk menentukan jumlah karakter yang harus tumpang tindih antara chunk yang berdekatan.

Jika anda ingin melihat ilustrasi tentang konfigurasi chunk, anda dapat melihatnya [di sini](https://dev.to/peterabel/what-chunk-size-and-chunk-overlap-should-you-use-4338).

Selanjutnya kita akan coba terapkan proses pemisahan kalimat ke dokumen yang kita gunakan.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(type(chunks[0]))

22
<class 'langchain_core.documents.base.Document'>


In [ ]:
# Mengecek hasil pemecahan menjadi chunk
print(f"Jumlah chunks yang dihasilkan: {len(chunks)}")
print(f"Tipe data chunk pertama: {type(chunks[0])}")

Jumlah chunks yang dihasilkan: 22
Tipe data chunk pertama: <class 'langchain_core.documents.base.Document'>


In [ ]:
type(chunks)

list

In [ ]:
list_python = [1,2,3,4,5]

In [ ]:
chunks[0].page_content

'Copyright © 2025 Pearson Education, Inc. or its affiliates(s). All rights reserved.\n1. AI Problem Definition\n1.1 Identify the problem you are trying to solve using AI (e.g., user \nsegmentation, improving customer service)\n• Identify the need that will be addressed'

In [ ]:
chunks[20].page_content

'• Tune the model by changing parameters\n• Gather performance metrics for the model\n• Iterate as needed\n3.3 Select specific model after experimentation, avoiding \noverengineering\n• Consider cost, speed, and other factors in evaluating models'

In [ ]:
# Menampilkan chunk pertama
# panjang chunk
print(f"Panjang chunk pertama: {len(chunks[0].page_content)} karakter")
print("\nContoh chunk pertama:")
print(chunks[0].page_content)  # Memastikan bahwa setiap chunk memiliki konten

Panjang chunk pertama: 264 karakter

Contoh chunk pertama:
Copyright © 2025 Pearson Education, Inc. or its affiliates(s). All rights reserved.
1. AI Problem Definition
1.1 Identify the problem you are trying to solve using AI (e.g., user 
segmentation, improving customer service)
• Identify the need that will be addressed


## Proses ke 3 - Embedding

Membuat Embedding setelah melakukan Chunking.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/embedding-001")

In [ ]:
embedding_model.embed_query("halo nama saya sardi irfansyah")

[0.03521261364221573,
 0.01966172084212303,
 -0.07645478844642639,
 0.038710817694664,
 0.039510320872068405,
 0.023734871298074722,
 0.020717380568385124,
 -0.03705525025725365,
 -0.003651504870504141,
 0.004835625644773245,
 0.0193785447627306,
 0.017872123047709465,
 -0.07380163669586182,
 -0.040347784757614136,
 -0.056593429297208786,
 0.024514134973287582,
 0.0408610962331295,
 -0.04113294929265976,
 0.013859566301107407,
 -0.027282433584332466,
 -0.00962763000279665,
 0.006484580226242542,
 -0.010257336311042309,
 -0.058877650648355484,
 0.015292790718376637,
 0.006248072255402803,
 -0.010633985511958599,
 -0.06691750138998032,
 -0.0690121278166771,
 0.08077341318130493,
 0.026047147810459137,
 0.027229640632867813,
 -0.09377671033143997,
 0.011227676644921303,
 -0.01789574697613716,
 -0.019316978752613068,
 0.019062280654907227,
 0.03901186212897301,
 0.05258435010910034,
 -0.03448620066046715,
 0.022909214720129967,
 -0.02484147809445858,
 0.006892734207212925,
 -0.068916752934

## Proses ke 4 - Vector Databases

Cek dokumentasi [vector store](https://python.langchain.com/docs/how_to/vectorstore_retriever/).

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
# Sematkan setiap chunk dan muat ke dalam chromadb
db = Chroma.from_documents(chunks, embedding_model, persist_directory="chroma_db")

# Menyimpan perubahan ke disk
db.persist()

In [ ]:
# mengatur koneksi untuk menghubungkan ke ChromaDB
db_connection = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)

In [ ]:
# Mengonversi koneksi Chroma menjadi objek retriever untuk pencarian dokumen berbasis vektor
# Langkah pertama dalam RAG - Retrieval Augmented Gen
retriever = db_connection.as_retriever(search_kwargs={"k": 10})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


`k` adalah parameter yang digunakan dalam fungsi as_retriever() untuk menentukan jumlah hasil pencarian teratas (top-k) yang akan diambil dari basis data vektor (Chroma database).

Parameter ini digunakan dalam proses pencarian dokumen relevan yang mengacu pada jumlah dokumen atau chunk yang akan diambil dari kumpulan dokumen yang lebih besar (misalnya, dari Chroma vector store) berdasarkan kesamaan atau kedekatannya dengan pertanyaan yang diajukan.

> Nilai k umumnya lebih kecil dari total chunk.



In [ ]:
check_response = retriever.invoke("whats skill needed for AI certification")
len(check_response)

10

In [ ]:
md(check_response[1].page_content)

security, and ethics of AI.  Successful candidates will be able to analyze and classify a problem. They should 
be able to demonstrate knowledge of data collection, data processing, and feature engineering strategies. 
Candidates should be able to choose an appropriate algorithm for training a model, and understand the 
metrics used to evaluate model performance. They should understand the AI development lifecycle and

## Langkah 5 Prepare RAG system

Dokumnetasi [Prompt templates](https://python.langchain.com/docs/concepts/prompt_templates/).

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
# Membuat template pesan untuk sistem dan pesan pengguna
# Proses ke 2 dalam RAG - Augmented
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""
    You are an AI that can answer questions based on context and user questions.
    You have to answer user questions, based on context.
    """),

    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""
    Answer the following questions based on the context.
    context: {context}
    question: {question}
    answer: """
    )
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
md(format_docs(check_response))

metrics used to evaluate model performance. They should understand the AI development lifecycle and 
how a production pipeline is used to allow for continuous improvement.  Candidates at least 150 hours of 
instruction and/or exploration of artificial intelligence methodology and solutions.

security, and ethics of AI.  Successful candidates will be able to analyze and classify a problem. They should 
be able to demonstrate knowledge of data collection, data processing, and feature engineering strategies. 
Candidates should be able to choose an appropriate algorithm for training a model, and understand the 
metrics used to evaluate model performance. They should understand the AI development lifecycle and

how a production pipeline is used to allow for continuous improvement.  Candidates at least 150 hours of 
instruction and/or exploration of artificial intelligence methodology and solutions.

• Set guidelines for algorithm selection from user perspective
• Consider how the subject of the data can interpret the results
• Consider out-of-context use of AI results
IT SPECIALIST EXAM OBJECTIVES
Artificial Intelligence
Candidates for this certification have a foundational knowledge the procedures used to develop an artificial 
intelligence (AI) solution, as well as an understanding of the issues surrounding the governance, transparency,

be able to demonstrate knowledge of data collection, data processing, and feature engineering strategies. 
Candidates should be able to choose an appropriate algorithm for training a model, and understand the

• Consider how the subject of the data can interpret the results
• Consider out-of-context use of AI results
IT SPECIALIST EXAM OBJECTIVES
Artificial Intelligence
Candidates for this certification have a foundational knowledge the procedures used to develop an artificial

intelligence (AI) solution, as well as an understanding of the issues surrounding the governance, transparency, 
security, and ethics of AI.  Successful candidates will be able to analyze and classify a problem. They should

Copyright © 2025 Pearson Education, Inc. or its affiliates(s). All rights reserved.
IT SPECIALIST EXAM OBJECTIVES
3. AI Algorithms and Models  
3.1 Consider applicability of specific algorithms 
• Evaluate AI algorithm families
• Decide which algorithms  are suitable, e.g., neural network, classification 
(like decision tree, k means)
3.2 Train a model using the selected algorithm 
• Train model for an algorithm with best-guess starting parameters.
• Tune the model by changing parameters

• Determine problem type (e.g., classification, regression, unsupervised, 
reinforcement)
1.3 Identify the areas of expertise needed to solve the problem
• Identify business expertise required
• Identify the need for domain (subject-matter) expertise on the problem
• Identify AI expertise needed

• Identify AI expertise needed
• Identify implementation expertise needed
1.4 Build a security plan
• Consider internal access levels or permissions
• Consider infrastructure security
• Assess the risk of using a certain model or potential attack surfaces (e.g.,

In [ ]:
# # Setelah dokumen diambil oleh retriever dan di-format
# formatted_docs = format_docs(chunks)  # Format dokumen yang dihasilkan dari text chunks

# # Cetak hasil format
# print("Hasil Format Docs:")
# print(formatted_docs)  # Menampilkan hasil setelah dokumen diformat

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)


In [ ]:
# Masuk proses ke 3 dalam RAG yaitu Generation
response = rag_chain.invoke("""
whats component in Maintaining and Monitoring AI in Production
""")

md(response)

Engage in oversight, Support the AI solution, Consider improvement or decommission on a regular basis.

In [ ]:
md(response)

Untuk mendapatkan sertifikasi IT Specialist: AI dari Pearson VUE, kandidat harus memiliki pemahaman dasar tentang:

* **Prosedur pengembangan solusi AI:**  Memahami langkah-langkah yang terlibat dalam siklus hidup pengembangan AI, termasuk pengumpulan data, pemrosesan data, rekayasa fitur, pemilihan algoritma, pelatihan model, evaluasi model, dan penerapan model.
* **Tata kelola, transparansi, keamanan, dan etika AI:** Memahami isu-isu seputar penggunaan AI yang bertanggung jawab, termasuk cara mengidentifikasi dan mengurangi potensi bias, memastikan keadilan, dan melindungi privasi.
* **Analisis dan klasifikasi masalah:** Kemampuan untuk menganalisis masalah dan mengklasifikasikannya untuk menentukan pendekatan AI yang tepat.
* **Strategi pengumpulan dan pemrosesan data:**  Memahami berbagai teknik pengumpulan data dan cara memproses data mentah agar sesuai untuk pelatihan model AI.
* **Rekayasa fitur:** Kemampuan untuk memilih, mengubah, atau membuat fitur yang relevan dari data mentah untuk meningkatkan kinerja model AI.
* **Pemilihan dan pelatihan algoritma:** Kemampuan untuk memilih algoritma yang sesuai untuk masalah tertentu dan melatih model menggunakan algoritma yang dipilih. Memahami berbagai famili algoritma AI, seperti jaringan saraf tiruan dan algoritma klasifikasi (misalnya, pohon keputusan, k-means). Menyetel parameter model untuk mengoptimalkan kinerjanya.
* **Metrik evaluasi model:** Memahami metrik yang digunakan untuk mengevaluasi kinerja model AI, seperti akurasi, presisi, dan recall.
* **Siklus hidup pengembangan AI dan pipeline produksi:** Memahami bagaimana model AI diintegrasikan dan di-deploy ke dalam sistem produksi dan bagaimana pipeline produksi digunakan untuk memungkinkan peningkatan berkelanjutan.
* **Dampak AI pada individu dan komunitas:** Kemampuan untuk menganalisis dan mengurangi dampak negatif potensial dari AI pada kelompok pengguna tertentu.
* **Masukan pengguna:** Kemampuan untuk mengumpulkan dan menggabungkan masukan pengguna untuk meningkatkan model AI.
* **Persyaratan sumber daya:** Kemampuan untuk mengidentifikasi dan menilai persyaratan sumber daya untuk proyek AI, seperti daya komputasi dan kompleksitas waktu.
* **Keamanan:** Kemampuan untuk mengembangkan rencana keamanan untuk solusi AI.
* **Penggunaan AI yang tepat:** Kemampuan untuk memastikan bahwa AI digunakan secara bertanggung jawab dan etis.
* **Dokumentasi keputusan data:** Kemampuan untuk mendokumentasikan asumsi, predikat, dan batasan yang mendasari pilihan desain dan menyediakan informasi ini kepada regulator dan pengguna akhir.


Selain pemahaman konseptual, kandidat juga harus memiliki setidaknya 150 jam instruksi dan/atau eksplorasi metodologi dan solusi kecerdasan buatan.

# Contoh lain

misalkan pada contoh ini kita akan mencoba menerapkan RAG
dengan `RecursiveCharacterTextSplitter` dan `PromptTemplate`.

In [ ]:
# Instalasi dependensi yang dibutuhkan
!pip install -q langchain PyPDF2 #python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
import os
import io
import PyPDF2
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
# from dotenv import load_dotenv
from google.colab import files
from IPython.display import Markdown as md

In [ ]:
# Membuka dan membaca file PDF
with open('/content/ai_pv.pdf', "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    pdf_pages = pdf_reader.pages

    # Mengekstrak teks dari semua halaman
    context = "\n\n".join(page.extract_text() for page in pdf_pages)

In [ ]:
context

'Copyright © 2023 Pearson Education, Inc. or its affiliate(s). All rights reserved. \n1. AI Problem Definition\n1.1 Identify the problem you are trying to solve using AI (e.g., user \nsegmentation, improving customer service)\n• Identify the need that will be addressed\n• Find out what information comes in and what output is expected\n• Determine whether AI is called for\n• Consider upsides and downsides of AI in the situation\n• Define measurable success\n• Benchmark against domain or organization-specific risks to which the \nproject may be susceptible\n1.2 Classify the problem (e.g., regression, unsupervised learning) \n• Examine available data (labeled or unlabeled?) and the problem\n• Determine problem type (e.g., classification, regression, unsupervised, \nreinforcement)\n1.3 Identify the areas of expertise needed to solve the problem\n• Identify business expertise required\n• Identify the need for domain (subject-matter) expertise on the problem\n• Identify AI expertise needed\n

In [ ]:
# Memecah teks menjadi potongan-potongan kecil
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_text(context)

In [ ]:
print(len(texts))

print(type(texts[0]))

27
<class 'str'>


In [ ]:
texts[0]

'Copyright © 2023 Pearson Education, Inc. or its affiliate(s). All rights reserved. \n1. AI Problem Definition\n1.1 Identify the problem you are trying to solve using AI (e.g., user \nsegmentation, improving customer service)\n• Identify the need that will be addressed\n• Find out what information comes in and what output is expected\n• Determine whether AI is called for\n• Consider upsides and downsides of AI in the situation\n• Define measurable success'

In [ ]:
# Membuat embeddings untuk potongan-potongan teks
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k": 20})

In [ ]:
vector_index

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x79b018b8c590>, search_kwargs={'k': 20})

In [ ]:
print(type(vector_index))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [ ]:
# Mendapatkan pertanyaan dari pengguna
user_question = input("Tanyakan pertanyaan: ")

# Mendapatkan dokumen relevan untuk pertanyaan pengguna
docs = vector_index.get_relevant_documents(user_question)

Tanyakan pertanyaan: jelaskan syarat sertifikasi AI di pearson vue


<ipython-input-41-ee8e5bac553b>:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vector_index.get_relevant_documents(user_question)


In [ ]:
# Mendefinisikan template prompt
prompt_template = """
Jawablah pertanyaan ini dengan se-detail mungkin dari konteks yang diberikan,
pastikan untuk memberikan semua detail, jika jawaban tidak ada dalam
konteks yang diberikan cukup katakan, "jawaban tidak tersedia dalam konteks",
jangan memberikan jawaban yang salah\n\n
Konteks:\n {context}?\n
Pertanyaan: \n{question}\n
Jawaban:
"""

# Membuat prompt
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

# Memuat QA chain
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", api_key=api_key)
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

# Mendapatkan jawaban dari model
response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)

# Menampilkan jawaban
print("\nJawaban:")
md(response['output_text'])

<ipython-input-42-ac923919d070>:17: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
<ipython-input-42-ac923919d070>:20: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)



Jawaban:


Syarat sertifikasi AI di Pearson Vue, berdasarkan konteks yang diberikan, meliputi:

**Pengetahuan dan Keterampilan Prasyarat:**

* Kemampuan membaca setara kelas 8.
* Pemahaman Aljabar I.
* Memahami bagaimana komunikasi terjadi pada jaringan.
* Keterampilan literasi digital, termasuk kemampuan untuk meneliti, membuat konten, dan memecahkan masalah menggunakan teknologi.
* Keterampilan berpikir komputasional, termasuk kemampuan untuk menguraikan masalah menjadi bagian-bagian yang lebih kecil dan memecahkan masalah melalui otomatisasi.

**Pengalaman:**

* Minimal 150 jam instruksi dan/atau eksplorasi metodologi dan solusi kecerdasan buatan.

**Keterampilan Tambahan yang Membantu (tidak wajib):**

* Kemampuan dalam setidaknya satu bahasa pemrograman atau scripting.
* Keakraban dengan teknologi penyimpanan data, termasuk basis data relasional.
* Keakraban dengan metode analisis data, termasuk statistik.
* Keakraban dengan prinsip-prinsip keamanan, termasuk CIA (kerahasiaan, identitas, dan ketersediaan) dan AAA (Autentikasi, Otorisasi, dan Akuntansi), serta penilaian risiko dan kerentanan.


Konteks tersebut menjelaskan detail tentang materi ujian,  tetapi tidak memberikan informasi tentang proses pendaftaran, biaya, atau persyaratan administrasi lainnya di Pearson Vue.  Informasi tersebut tidak tersedia dalam konteks.